### Problem 1 : Getting to Philosophy 

Lets play a variation of the [wiki game](https://en.wikipedia.org/wiki/Wikipedia:Wiki_Game) to learn about [this](https://en.wikipedia.org/wiki/Wikipedia:Getting_to_Philosophy) phenomenon. The rules are as follows: 
 - Start using the random article link (wiki menu on the left hand side)
 - Click on the first non-italicized link outside of parentheses 
 - Ignore external links (e.g., `/wiki/File:...` or `/wiki/Category:...`), links to the current page
 - Stop when reaching "Philosophy", a dead end (page with no links) or when a loop occurs

#### Exercise

Write a function `play` that plays the game and stops if "Philosophy" is not reached after `maxiter = 1000` steps. This function should return information to compute the quantities below. 

Play the game $200$ times. Report 
 - the mean number of sites visited per game, 
 - the maximum number of sites visited per game,
 - and number of convergences to "Philosophy" and 
 - the 20 most visited sites over all 200 games. 
 
You may want to use the module `lxml.html` and the function `tostring` `lxml.etree` or similar packages to to parse the html. Besides these, you are allowed to use `requests`, `re`, and `time`. To display the results, you may use `pandas` and its method `pandas.Series.value_counts()` or similar packages. You might find [regexr.com](https://regexr.com/) helpful. 

__Hint:__ Consider the results below from the function `play`, which takes the wiki-style url as argument and returns a dictionary. 

In [1]:
import lxml.html as lx
import requests
from lxml.etree import tostring
import re 
import tim



In [2]:
sim(1) # takes ~ 20 min, use sim(1) for a single result

[['/wiki/Gardena_elkinsi',
  '/wiki/Thread-legged_bug',
  '/wiki/Reduviidae',
  '/wiki/Cosmopolitan_distribution',
  '/wiki/Biogeography',
  '/wiki/Species_distribution',
  '/wiki/Taxon',
  '/wiki/Biology',
  '/wiki/Science',
  '/wiki/Scientific_method',
  '/wiki/Empirical_evidence',
  '/wiki/Proposition',
  '/wiki/Philosophy_of_language',
  '/wiki/Analytic_philosophy',
  '/wiki/Philosophy'],
 [15],
 ['philosophy']]

In [6]:
import pandas as pd 
print(pd.Series(result[1]).mean()) 

18.595


In [7]:
print(pd.Series(result[1]).max())

31


In [8]:
pd.Series(result[2]).value_counts()

philosophy    186
loop           13
deadend         1
dtype: int64

In [9]:
pd.Series(result[0]).value_counts().head(20)

/wiki/Philosophy                186
/wiki/Outline_of_philosophy     167
/wiki/Epistemology              167
/wiki/Descriptive_knowledge     167
/wiki/Knowledge                 167
/wiki/Academic_discipline       143
/wiki/Analytic_philosophy       142
/wiki/Philosophy_of_language    142
/wiki/Proposition               142
/wiki/Empirical_evidence        142
/wiki/Scientific_method         139
/wiki/Science                   135
/wiki/Branches_of_science        59
/wiki/Psychology                 42
/wiki/Politics                   31
/wiki/Decision-making            31
/wiki/Social_science             31
/wiki/Physics                    28
/wiki/Natural_science            28
/wiki/Social_group               26
dtype: int64